In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers

In [5]:
data = pd.read_csv("DataSet2.csv", sep=";", parse_dates=['Timestamp']) #, nrows=10)

In [6]:
data.head(5)

,Timestamp,ARO,SATUR_PRESSURE,BOIL_PERCENT,OCTANE_NUMBER,percent_01_LAB,percent_04_LAB,quality_01_LAB,pressure_01_LAB,consumption_09,...,temperature_08,temperature_14,temperature_01_LAB,temperature_21,temperature_22,temperature_23,temperature_24,consumption_01,pressure_11,temperature_20
0,2016-01-01 00:00:00,65.59,48.0,6.0,98.9,6.87640,64.05901,98.45841,56.06648,0.00292,...,134.7649,25.61035,90.86990,-3.72980,-7.13281,-1.0,11.86261,59.14596,70.77777,519.1353
1,2016-01-01 00:01:00,65.59,48.0,6.0,98.9,6.86923,64.01093,98.45134,56.04396,0.00292,...,134.8389,25.61035,90.90942,-3.72989,-7.13306,-1.0,11.86295,59.18106,70.93322,519.1353
2,2016-01-01 00:02:00,65.59,48.0,6.0,98.9,6.86095,63.99218,98.44153,56.01728,0.00292,...,134.6192,25.61055,90.88013,-3.73050,-7.13440,-1.0,11.86490,59.16980,70.03181,519.1346
3,2016-01-01 00:03:00,65.59,48.0,6.0,98.9,6.85725,63.98800,98.44115,56.01207,0.00292,...,134.6191,25.63477,90.87057,-3.73145,-7.13660,-1.0,11.86804,59.30387,70.46443,519.1337
4,2016-01-01 00:04:00,65.59,48.0,6.0,98.9,6.85561,63.98110,98.43896,56.00104,0.00292,...,134.6191,25.63477,90.90518,-3.73212,-7.13818,-1.0,11.87030,59.26101,69.81942,519.1330


In [7]:
def preprocess(df, K=5):
    df = df.groupby(df.index //5).mean() #усреднение
    df_np = df.to_numpy()
    df_np = df_np[1:len(df_np), ...] - df_np[0:-1, ...] #дифференцирование
    df_np = (df_np - df_np.mean()) / df_np.std() #масштабирование
    df = pd.DataFrame(df_np, columns=df.columns)
    return df

In [8]:
#усреднение данных за 5 минут, дискретное дифференцирование, масштабирование
preprocess(data).to_csv("DataSet_preprocessed.csv")